# Прогнозирование оттока клиентов
Data from https://www.kaggle.com/blastchar/telco-customer-churn

In [67]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [68]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
len(df)

7043

## Подготовка исходных данных

In [69]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [70]:
df.head().T # еще вариант отображения, но умещается на все

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [71]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

SeniorCitizen имеет тип integer , а TotalCharges - не идентифицирован , как числовой объект

In [72]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') # преобразуем в числовой тип данных

In [73]:
df['TotalCharges'].dtypes

dtype('float64')

In [74]:
df[df['TotalCharges'].isnull()][['customerID', 'TotalCharges']] # видим, что есть пропуски

,customerID,TotalCharges
488,4472-LVYGI,NaN
753,3115-CZMZD,NaN
936,5709-LVOEQ,NaN
1082,4367-NUYAO,NaN
1340,1371-DWPAZ,NaN
3331,7644-OMVMY,NaN
3826,3213-VVOLG,NaN
4380,2520-SGTTA,NaN
5218,2923-ARZLG,NaN
6670,4075-WKNIU,NaN


In [75]:
df['TotalCharges'] = df['TotalCharges'].fillna(0) # заменяем нулями

In [76]:
# ПОНИЖАЕМ РЕГИСТР, ЗАМЕНЯЕМ ПРОБЕЛЫ СИМВОЛАМИ ПОДЧЕРКИВАНИЯ
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [77]:
df.churn = (df.churn == 'yes').astype(int) #  с помощью двоичной классификации перекодируем из boolean yes в значение 1, no - в значение 0

In [78]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


## Разбиваем подвыборку на трейн и тест

In [79]:
from sklearn.model_selection import train_test_split

In [80]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1) # сплитим на полный обучающий и тестовый

In [81]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11) # сплитим полный обучающий на обучающий и валидационный

In [82]:
# Принимает столбец с целевой переменной churn и сохраняет его за пределами датафрейма
y_train = df_train.churn.values
y_val = df_val.churn.values

In [83]:
# Удаляем столбцы churn из обоих df для гарантии того, что мы случайно не используем переменную churn в качестве признака при обучении
del df_train['churn']
del df_val['churn']

## Исследовательский анализ данных

In [84]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [85]:
# Проверяем распределение значений целевой переменной
df_train_full.churn.value_counts()

churn
0    4113
1    1521
Name: count, dtype: int64

In [86]:
# Зная абсолютные значения, можем найти долю

global_mean = df_train_full.churn.mean().item()
round(global_mean, 3)

0.27

In [87]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [88]:
df_train_full[categorical].nunique() # считаем уникальные значения по каждой колонке

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## Важность признака - 
понимание того, как другие переменные влияют на целевую

In [89]:
# КОЭФИЦИЕНТ ОТТОКА
# Рассмотрим влияние переменной Gender на отток (churn)
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.277
gender == male:   0.263


Глобальные коэф оттока 0.27 , в то время как цифры по обоим полам довольно похожи. Это означает, что gender не является полезной переменной при прогнозировании оттока

In [90]:
# КОЭФИЦИЕНТ РИСКА для переменной gender:
print(f'Коэфициент риска для Gender == женщина: {female_mean / global_mean}')
print(f'Коэфициент риска для Gender == мужчина: {male_mean / global_mean}')

Коэфициент риска для Gender == женщина: 1.0253955354648652
Коэфициент риска для Gender == мужчина: 0.9749802969838747


In [91]:
# КОЭФИЦИЕНТ ОТТОКА для показателя partner:
# Рассмотрим влияние переменной partner на отток (churn)

partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
print('partner == yes:', round(partner_yes, 3))

partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print('partner == no :', round(partner_no, 3))

partner == yes: 0.205
partner == no : 0.33


In [92]:
# КОЭФИЦИЕНТ РИСКА для переменной partner:
print(f'Коэфициент риска для partner == Yes: {partner_yes / global_mean}')
print(f'Коэфициент риска для partner == No: {partner_no / global_mean}')

Коэфициент риска для partner == Yes: 0.7594724924338315
Коэфициент риска для partner == No: 1.2216593879412643


In [93]:
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


Как мы видим, оба показателя (коэф оттока (mean) и коэффициент риска невелики для обоих полов). Поэтому, влияние отдельных переменных на отток невелико

In [94]:
df_group = df_train_full.groupby(by='partner').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


С другой стороны, уровень риска для людей, у которых нет партнера значительно выше среднего. Люди с партнерами, как правило, меньше подвержены оттоку.

In [95]:
from IPython.display import display

In [96]:
# вычисляем средний отток
global_mean = df_train_full.churn.mean().item()
global_mean

0.26996805111821087

In [97]:
# Циклы по всем категориальным переменным:
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean']) # выполняет groupby для каждой категориальной переменной
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


Что касается пола, то между мужчинами и женщинами нет большой разницы в контексте оттока. Оба средних примерно одинаковы, и для обеих групп риски близки к 1

Пожилые люди склонны к оттоку больше, чем те, кто не является пенсионером. Риск оттока составляет примерно 1,53 для пожилых людей и 0,89 для остальных.

У людей, имеющих партнера, отток меньше, чем у людей без партнера. Риски составляют 0.75 и 1.22 соответственно.

Люди, пользующиеся телефонной связьюб не подвержены риску оттока: риск оттока близок к 1, и почти нет разницы в сравнении с общим уровнем оттока. Люди, которые не пользуются телефонной связью, еще менее склонны к оттоку: риск ниже 1, а разница с общим показателем оттока отрицательна.

ВЗАИМНАЯ ИНФОРМАЦИЯ - для категориальных переменных (Степень зависимости между категориальной и целевой переменной):

In [98]:
# ВЗАИМНАЯ ИНФОРМАЦИЯ - для категориальных переменных (Степень зависимости между категориальной и целевой переменной):
# Если переменные зависимы, то значения одной переменной дает нам некоторую информацию о другой. 
# С другой стороны, если переменная полностью независима от целевой переменной, то она является бесполезной, и может быть удалена из набора данных
# Чем выше степень зависимости, тем полезнее признак

from sklearn.metrics import mutual_info_score

def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


В верхней таблице представлены наиболее полезные признаки, в нижней - наименее

КОЭФФИЦИЕНТ КОРРЕЛЯЦИИ (значение от -1 до 1):
1) Положительная означает, что когда одна переменная увеличивается, другая также имеет тенденцию к увеличению.
2) Нулевая корреляция означает отсутствие связи между двумя переменными (те они полностью независимы)
3) Отрицательная корреляция возникает, когда одна переменная увеличивается, другая уменьшается

In [99]:
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')

,correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


In [100]:
df_train_full.groupby(by='churn')[numerical].mean()

,tenure,monthlycharges,totalcharges
churn,,,
0,37.531972,61.176477,2548.021627
1,18.070348,74.521203,1545.689415


# Конструирование признаков

## Прямое кодирование категориальных переменных (One-hot-encoding)

In [101]:
from sklearn.feature_extraction import DictVectorizer

In [102]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [103]:
train_dict[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'tenure': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9}

In [104]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [105]:
X_train = dv.transform(train_dict)

In [106]:
X_train.shape

(3774, 45)

In [107]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

# Обучение логистической регрессии

In [108]:
from sklearn.linear_model import LogisticRegression

In [109]:
# ОБУЧАЕМ:
model = LogisticRegression(
    solver='liblinear', # базовая библиотека оптимизации
    random_state=1) # начальное значение для генератора случайных чисел, необходим для перетасовки данных при обучении

model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [110]:
# ПРЯМОЕ КОДИРОВАНИЕ ВСЕХ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ:

val_dict = df_val[categorical + numerical].to_dict(orient='records') 
X_val = dv.transform(val_dict) # получаем матрицу признаков

In [111]:
# Получаем вероятности:
model.predict_proba(X_val)

array([[0.7650904 , 0.2349096 ],
       [0.73113804, 0.26886196],
       [0.6805489 , 0.3194511 ],
       ...,
       [0.94274833, 0.05725167],
       [0.38477034, 0.61522966],
       [0.93872731, 0.06127269]])


В левом столбце - вероятность того, что наблюдение относится к отрицательному классу (т.е клиент не уйдет)

В правом столбце - вероятность того, что наблюдение относится к положительному классу (т.е клиент уйдет)


In [112]:
# По сути столбцы содержат одну и ту же информацию, выбираем второй столбец
y_pred = model.predict_proba(X_val)[:, 1]

In [113]:
y_pred

array([0.2349096 , 0.26886196, 0.3194511 , ..., 0.05725167, 0.61522966,
       0.06127269])

In [114]:
# Делаем отсечку, чтобы получить твердые прогнозы (True or False)

churn = y_pred > 0.5

In [115]:
# Расчитываем достоверность (оценивая прогноз с фактом и подсчитывая среднее значение получившегося массива Bulean):

(y_val == churn).mean().item()

0.8016129032258065

Данный показатель означает, что прогнозы модели соответствовали фактическим в 80% случаев

# Интерпретация модели

In [116]:
# Получаем компонент смещения:

model.intercept_[0].item()

-0.1219885045100661

In [117]:
# Узнаем, какой признак связан с каждым веесом:

dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': np.float64(0.563),
 'contract=one_year': np.float64(-0.086),
 'contract=two_year': np.float64(-0.599),
 'dependents=no': np.float64(-0.03),
 'dependents=yes': np.float64(-0.092),
 'deviceprotection=no': np.float64(0.1),
 'deviceprotection=no_internet_service': np.float64(-0.116),
 'deviceprotection=yes': np.float64(-0.106),
 'gender=female': np.float64(-0.027),
 'gender=male': np.float64(-0.095),
 'internetservice=dsl': np.float64(-0.323),
 'internetservice=fiber_optic': np.float64(0.317),
 'internetservice=no': np.float64(-0.116),
 'monthlycharges': np.float64(0.001),
 'multiplelines=no': np.float64(-0.168),
 'multiplelines=no_phone_service': np.float64(0.127),
 'multiplelines=yes': np.float64(-0.081),
 'onlinebackup=no': np.float64(0.136),
 'onlinebackup=no_internet_service': np.float64(-0.116),
 'onlinebackup=yes': np.float64(-0.142),
 'onlinesecurity=no': np.float64(0.258),
 'onlinesecurity=no_internet_service': np.float64(-0.116),
 'onlinesecurity=yes':

In [118]:
# Чтобы облегчить восприяте модели, строим упрощенную модель, а далее повторяем те же шаги обучения:

	
subset = ['contract', 'tenure', 'totalcharges']
train_dict_small = df_train[subset].to_dict(orient='records')
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)

#
X_small_train = dv_small.transform(train_dict_small)

dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'tenure', 'totalcharges'], dtype=object)

In [119]:
# Обучим на этом наборе признаков нашу небольшую модель:

model_small = LogisticRegression(solver='liblinear', random_state=1)
model_small.fit(X_small_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [120]:
# Проверяем компонент смещения:
model_small.intercept_[0].item()

-0.5772299097126418

In [121]:
# Проверяем другие веса признаков:

dict(zip(dv_small.get_feature_names_out(), model_small.coef_[0].round(3)))

{'contract=month-to-month': np.float64(0.866),
 'contract=one_year': np.float64(-0.327),
 'contract=two_year': np.float64(-1.117),
 'tenure': np.float64(-0.094),
 'totalcharges': np.float64(0.001)}

Компонент смещения (-0.577) - Это базовый прогноз или оценка, которую мы бы получили в среднем. Далее откладываем перпендикуляр от этого значения по оси X и ищем точку соприкосновения с сигмоидой. По оси Y -вероятность оттока. Итоговая вероятность составляет менее 0.5, поэтому средний клиент, скорее всего, не расторгнет договор.

Для остальных признаков мы применяем скалярное произведение, складываем полученные значения всех признаков с компонентом смещения и на выходе получаем итоговое значение. На этом значении также откладываем перпендикуляр по оси X, находим точку соприкосновения с сигмоидой, далее на оси Y находим итоговую вероятность оттока

In [122]:
val_dict_small = df_val[subset].to_dict(orient='records')
X_small_val = dv_small.transform(val_dict_small)

In [123]:
y_pred_small = model_small.predict_proba(X_small_val)[:, 1]

In [124]:
y_pred_small

array([0.3872519 , 0.23836334, 0.15069922, ..., 0.12069581, 0.5501661 ,
       0.03389633])

# Использование модели

In [125]:
# Берем клиента, которого хотим оценить, помещаем все значения переменных в словарь
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}

In [126]:
# Преобразуем словарь в матрицу
X_test = dv.transform([customer])

In [127]:
# Берем эту матрицу и отправляем в обученную модель

model.predict_proba(X_test)

array([[0.92667668, 0.07332332]])

In [128]:
# На выходе получаем матрицу с прогнозами
print(list(X_test[0]))

[np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(79.85), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(41.0), np.float64(3320.75)]


In [129]:
X_test_values = [float(value) for value in X_test[0]]
print(X_test_values)

[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 79.85, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 41.0, 3320.75]


In [130]:
# Число в первом столбце во второй строке - вероятность оттока для клиента
model.predict_proba(X_test)[0, 1].item()

0.0733233241408035

In [131]:
# Повторяем на другом клиенте:

customer = {
    'gender': 'female',
    'seniorcitizen': 1,
    'partner': 'no',
    'dependents': 'no',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'onlinesecurity': 'no',
    'onlinebackup': 'no',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 85.7,
    'totalcharges': 85.7
}

In [132]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1].item()

0.8321646331247229